# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [10]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.216302366534234
 1.4863981313327952
 0.7659562145597467

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
```
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [11]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0        0.0       0.0
 0.559902   1.0       0.0
 0.425301  -0.531314  1.0
U factor:
3×3 Array{Float64,2}:
 0.463146  0.630137   0.393114
 0.0       0.105937  -0.172217
 0.0       0.0        0.54892

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0       0.0      0.0
 0.199621  1.0      0.0
 0.528877  0.20875  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.542519  0.906374   0.325869
 0.0       0.488986  -0.0453092
 0.0       0.0        0.462266

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [12]:
A\b

3-element Array{Float64,1}:
 0.9999999999999998
 1.0000000000000002
 1.0

In [13]:
Alu\b

3-element Array{Float64,1}:
 0.9999999999999998
 1.0000000000000002
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [14]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
```

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`.

In [16]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.34791    0.569959   0.744382
 -0.818033   0.203342  -0.538028
 -0.458018  -0.796115   0.395501
R factor:
3×3 Array{Float64,2}:
 -0.566171  -0.799247  -0.624491
  0.0       -0.116418   0.502119
  0.0        0.0        0.408606

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [17]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.34791    0.569959   0.744382
 -0.818033   0.203342  -0.538028
 -0.458018  -0.796115   0.395501

In [18]:
Aqr.R

3×3 Array{Float64,2}:
 -0.566171  -0.799247  -0.624491
  0.0       -0.116418   0.502119
  0.0        0.0        0.408606

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [19]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
values:
3-element Array{Float64,1}:
 -0.8574585429773882
  0.2414468057720227
  2.3660160418773963
vectors:
3×3 Array{Float64,2}:
 -0.604172   0.611874  -0.510476
 -0.124095  -0.705046  -0.69822
  0.787132   0.358497  -0.501899

The values and the vectors can be extracted from the Eigen type by special indexing

In [20]:
AsymEig.values

3-element Array{Float64,1}:
 -0.8574585429773882
  0.2414468057720227
  2.3660160418773963

In [21]:
AsymEig.vectors

3×3 Array{Float64,2}:
 -0.604172   0.611874  -0.510476
 -0.124095  -0.705046  -0.69822
  0.787132   0.358497  -0.501899

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [22]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0           2.88658e-15  -6.85563e-15
 -6.66134e-16   1.0           4.08007e-15
 -1.11022e-15  -2.35922e-16   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [25]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [26]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [27]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.9768634708331494

In [28]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [29]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [32]:
@time eigvals(Asym);

  0.083143 seconds (12 allocations: 7.989 MiB)


In [36]:
@time eigvals(Asym_noisy);

  0.727460 seconds (14 allocations: 7.921 MiB)


In [37]:
@time eigvals(Asym_explicit);

  0.074525 seconds (6.40 k allocations: 8.334 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [38]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.907364 seconds (473.38 k allocations: 206.833 MiB, 22.34% gc time)


6.902054572434398

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [39]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [40]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 1//10  3//10  1//5
 2//5   9//10  7//10
 1//10  7//10  2//5

In [41]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
 3//5
 2//1
 6//5

In [42]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [43]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1  0//1   0//1
 1//4  1//1   0//1
 1//4  3//19  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 2//5   9//10   7//10
 0//1  19//40   9//40
 0//1   0//1   -1//95

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [44]:
using LinearAlgebra

In [48]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
A_eigv = eigvals(A)

5-element Array{Float64,1}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [49]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [85]:
A_diag = Matrix(Diagonal(A_eigv))

5×5 Array{Float64,2}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [94]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [97]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0      0.0      0.0      0.0     542.468]
# #### 11.3
# Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]